# Welcome to ExKaldi

In this section, we will decode the test data based on HMM-DNN model and WFST graph.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Restorage the posteiori probability of AM from file (generated in 11_train_DNN_acoustic_model_with_tensorflow).

In [ ]:
probFile = os.path.join(dataDir, "exp", "train_DNN", "amp.npy")

prob = exkaldi.load_prob(probFile)

prob

In [ ]:
prob.subset(nHead=1).data

As above, this is naive output without log softmax activation function. We need do softmax.

Exkaldi Numpy achivements have a method __.map(...)__ . We map a softmax function to all matrixs.

In [ ]:
prob = prob.map( lambda x: exkaldi.nn.log_softmax(x, axis=1) )

prob.subset(nHead=1).data

Then decode based on WFST. HCLG graph file and HMM model file have been generated (07_train_triphone_HMM-GMM_delta).

In [ ]:
HCLGFile = os.path.join(dataDir, "exp", "train_delta", "graph", "HCLG.fst")

hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")

And for convenience, prepare lexicons.

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

Use __nn_decode__ function. 

In [ ]:
lat = exkaldi.decode.wfst.nn_decode(prob, hmmFile, HCLGFile, symbolTable=lexicons("words"))

lat

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_DNN", "decode_test")

exkaldi.utils.make_dependent_dirs(outDir, False)

lat.save( os.path.join(outDir,"test.lat") )

From lattice get 1-best result and score it.

In [ ]:
refIntFile = os.path.join(dataDir, "exp", "train_delta", "decode_test", "text.int")

for penalty in [0., 0.5, 1.0]:
    for LMWT in range(10, 15):
    
        newLat = lat.add_penalty(penalty)
        result = newLat.get_1best(lexicons("words"), hmmFile, lmwt=LMWT, acwt=0.5)

        score = exkaldi.decode.score.wer(ref=refIntFile, hyp=result, mode="present")
        
        print(f"Penalty {penalty}, LMWT {LMWT}: WER {score.WER}")

In step 10_process_lattice_and_score, the best WER based on HMM-GMM model is about 135% and here is 107% in our experiment. So it is a truth that the performance of HMM-DNN got better.

Up to here, the simple tutorial is over.